In [21]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import START, END, StateGraph
from pydantic import BaseModel, Field
from typing import List, TypedDict, Literal
import operator

# Load API Key (Ensure your .env file has OPENAI_API_KEY)

# Initialize the LLM
# llm = ChatOpenAI(
#     model="gpt-4o-mini", # or "gpt-3.5-turbo" / "gpt-4"
#     temperature=0
# )
load_dotenv()
google_api_key = os.getenv("GEMNI_API_KEY")

if not google_api_key:
    raise ValueError("GEMNI_API_KEY not found! Please set it in your .env file.")

print("API key loaded")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite", temperature=0.3, api_key=google_api_key
)

API key loaded


In [27]:
# 1. Define the Metric Structure using standard Pydantic
class QualityMetrics(BaseModel):
    clarity: int = Field(..., description="Score from 1-5 indicating how clear and understandable the text is.")
    completeness: int = Field(..., description="Score from 1-5 indicating if all parts of the prompt are addressed.")
    accuracy: int = Field(..., description="Score from 1-5 indicating factual correctness and precision.")
    critique: str = Field(..., description="Specific constructive feedback on how to improve.")

# 2. Define the Agent State
class AdaptiveReflectionState(TypedDict):
    task: str
    draft: str
    metrics_history: List[QualityMetrics]  # Tracks history of scores
    iterations: int

MAX_ITERATIONS = 3

In [30]:

# Node 1: Generator
def generator_node(state: AdaptiveReflectionState) -> dict:
    task = state["task"]
    
    # Check if this is a first draft or a refinement
    if not state["metrics_history"]:
        # First draft
        prompt = f"""Write a response for the following task:
        
Task: {task}

Provide a clear, complete, and accurate answer."""
        print(f"\n Generating initial draft...")
        
    else:
        # Refinement based on specific scores
        latest_metrics = state["metrics_history"][-1]
        draft = state["draft"]
        prompt = f"""Improve the draft based on the following critique and scores (aim for 5/5):
        
Original Task: {task}
Current Draft: {draft}

Critique: {latest_metrics.critique}
Scores - Clarity: {latest_metrics.clarity}/5, Completeness: {latest_metrics.completeness}/5, Accuracy: {latest_metrics.accuracy}/5

Refine the text to improve these specific areas."""
        print(f"\nRefining draft (Iteration {state['iterations'] + 1})...")

    response = llm.invoke([HumanMessage(content=prompt)])
    return {"draft": response.content, "iterations": state["iterations"] + 1}

# Node 2: Critic (Structured)
def critic_node(state: AdaptiveReflectionState) -> dict:
    # Bind the Pydantic model to the LLM for structured output
    structured_critic = llm.with_structured_output(QualityMetrics)
    
    prompt = f"""Evaluate the following text based on the task.
       
Task: {state['task']}
Text: {state['draft']}

Score the text 1-5 on Clarity, Completeness, and Accuracy. Provide constructive feedback."""
    
    print(" Critiquing...")
    metrics: QualityMetrics = structured_critic.invoke([HumanMessage(content=prompt)])
    
    # Print current scores for visibility
    print(f"   Scores -> Clarity: {metrics.clarity}, Completeness: {metrics.completeness}, Accuracy: {metrics.accuracy}")
    
    # Append new metrics to history
    return {"metrics_history": [metrics]}

# Logic: Stop only if ALL scores are >= 4 or max iterations reached
def should_continue(state: AdaptiveReflectionState) -> Literal["generator", "end"]:
    latest_metrics = state["metrics_history"][-1]
    
    # Stop conditions
    if state["iterations"] >= MAX_ITERATIONS:
        print(" Max iterations reached.")
        return "end"
    
    # Quality Check: If ALL scores are >= 4, we are done
    if (latest_metrics.clarity >= 4 and 
        latest_metrics.completeness >= 4 and 
        latest_metrics.accuracy >= 4):
        print(" Quality standards met!")
        return "end"
    
    # Otherwise, refine again
    return "generator"

In [31]:
builder = StateGraph(AdaptiveReflectionState)

builder.add_node("generator", generator_node)
builder.add_node("critic", critic_node)

builder.add_edge(START, "generator")
builder.add_edge("generator", "critic")
builder.add_conditional_edges(
    "critic",
    should_continue,
    {
        "generator": "generator",
        "end": END
    }
)

adaptive_agent = builder.compile()

In [32]:
def visualize_history(result):
    print(f"\n{'='*20} SCORING HISTORY {'='*20}")
    print(f"{'Iter':<5} | {'Clarity':<8} | {'Comp.':<8} | {'Acc.':<8} | {'Status'}")
    print("-" * 55)
    
    for i, m in enumerate(result["metrics_history"]):
        status = "Refine" if (m.clarity < 4 or m.completeness < 4 or m.accuracy < 4) else "Pass"
        print(f"{i+1:<5} | {m.clarity:<8} | {m.completeness:<8} | {m.accuracy:<8} | {status}")
    print("-" * 55)
    print(f"Final Output:\n{result['draft'][:100]}...")

# Run the test
query = "explain quantum physics"

print(f"Starting Adaptive Reflection on: '{query}'")
result = adaptive_agent.invoke({
    "task": query,
    "draft": "",
    "metrics_history": [],
    "iterations": 0
})

visualize_history(result)

Starting Adaptive Reflection on: 'explain quantum physics'

 Generating initial draft...
 Critiquing...
   Scores -> Clarity: 5, Completeness: 5, Accuracy: 5
 Quality standards met!

==================== SCORING HISTORY ====================
Iter  | Clarity  | Comp.    | Acc.     | Status
-------------------------------------------------------
1     | 5        | 5        | 5        | Pass
-------------------------------------------------------
Final Output:
## Quantum Physics: A Journey into the Bizarre and Wonderful World of the Very Small

Quantum physic...


In [34]:
query = "Write a professional business proposal to microsoft requesting a for a branch in Abeokuta"

print(f"Starting Adaptive Reflection on: '{query}'")
result = adaptive_agent.invoke({
    "task": query,
    "draft": "",
    "metrics_history": [],
    "iterations": 1
})

visualize_history(result)

Starting Adaptive Reflection on: 'Write a professional business proposal to microsoft requesting a for a branch in Abeokuta'

 Generating initial draft...
 Critiquing...
   Scores -> Clarity: 5, Completeness: 5, Accuracy: 5
 Quality standards met!

==================== SCORING HISTORY ====================
Iter  | Clarity  | Comp.    | Acc.     | Status
-------------------------------------------------------
1     | 5        | 5        | 5        | Pass
-------------------------------------------------------
Final Output:
## Business Proposal: Establishing a Microsoft Branch in Abeokuta, Nigeria

**To:**
The Executive Le...
